In [2]:
import deepchem as dc
import numpy as np

## Chapter 3

In [46]:
x = np.random.random((4,5))
y = np.random.random((4,1))

In [47]:
dataset = dc.data.NumpyDataset(x,y)

In [48]:
print(dataset.y)

[[0.29644818]
 [0.55550374]
 [0.396535  ]
 [0.87232197]]


In [49]:
tox21_tasks, tox21_datasets,transformers = dc.molnet.load_tox21()

Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [50]:
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [51]:
model = dc.models.MultitaskClassifier(n_tasks = 12,
                                     n_features=1024,
                                     layer_sizes=[1000])

In [52]:
model.fit(train_dataset,nb_epoch=10)

863.4335116431827

In [53]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score,np.mean)

In [54]:
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)

computed_metrics: [0.9916344997926803, 0.9961865767501605, 0.9615471242679994, 0.9824871044619803, 0.906487361060319, 0.9842916942057721, 0.9919977278662133, 0.9033483132903182, 0.9881340346457133, 0.9721711222524851, 0.9469647766089786, 0.9751132740392684]
computed_metrics: [0.7674229846891718, 0.8563480020232677, 0.8993709350677044, 0.8025434373037471, 0.7084468071199808, 0.8004004004004004, 0.7242634889109567, 0.7209362603678326, 0.8589769716731199, 0.7136618325642716, 0.8704896907216495, 0.7827050997782705]


### MNIST Model

In [58]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [60]:
import deepchem as dc
import tensorflow as tf
import deepchem.models.tensorgraph.layers as layers

In [61]:
train_dataset = dc.data.NumpyDataset(mnist.train.images,mnist.train.labels)
test_dataset = dc.data.NumpyDataset(mnist.test.images,mnist.test.labels)

In [63]:
model = dc.models.TensorGraph(model_dir='mnist')
feature = layers.Feature(shape=(None,784))
label = layers.Label(shape=(None,10))

In [64]:
# Reshape for 28x28 for convolutional network:
make_image = layers.Reshape(shape=(None,28,28),in_layers=feature)

In [65]:
conv2d_1 = layers.Conv2D(num_outputs=32, activation_fn = tf.nn.relu,
                        in_layers = make_image)
conv2d_2 = layers.Conv2D(num_outputs=64, activation_fn = tf.nn.relu,
                        in_layers = conv2d_1)

In [67]:
flatten = layers.Flatten(in_layers=conv2d_2)
dense1 = layers.Dense(out_channels=1024,activation_fn=tf.nn.relu, in_layers=flatten)
dense2 = layers.Dense(out_channels=10,activation_fn=None, in_layers=dense1)

In [69]:
smce = layers.SoftMaxCrossEntropy(in_layers=[label,dense2])
loss = layers.ReduceMean(in_layers=smce)
model.set_loss(loss)

In [70]:
output = layers.SoftMax(in_layers=dense2)
model.add_output(output)

In [ ]:
model.fit(train_dataset, nb_epoch=10)